In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_lightning
import os
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
import PIL
from tqdm import tqdm
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 32.7 MB/s eta 0:00:00


In [ ]:
class ResNext(pl.LightningModule):

  def __init__(self, model_variant, num_classes):
    super().__init__()
    self.model_variant = model_variant
    if self.model_variant=='50_32x4d':
      self.model = models.resnext50_32x4d(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_variant=='101_32x8d':
      self.model = models.resnext101_32x8d(weights=None)
      self.features = self.model.fc.in_features
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_variant=='101_64x4d':
      self.model = models.resnext101_64x4d(weights=None)
      self.features = self.model.fc.in_features
      self.model.fc = nn.Linear(self.features,num_classes)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), weight_decay=0.0001, momentum=0.9, lr=0.1)

class ResNext_Model(pl.LightningModule):

  def __init__(self, model_variant, num_classes, w1=1, w2=0):
    super().__init__()
    self.model_variant = model_variant
    self.w1 = w1
    self.w2 = w2
    if self.model_variant=='50_32x4d':
      self.model = models.resnext50_32x4d(weights=None)
      self.features = self.model.fc.in_features 
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_variant=='101_32x8d':
      self.model = models.resnext101_32x8d(weights=None)
      self.features = self.model.fc.in_features
      self.model.fc = nn.Linear(self.features,num_classes)
    if self.model_variant=='101_64x4d':
      self.model = models.resnext101_64x4d(weights=None)
      self.features = self.model.fc.in_features
      self.model.fc = nn.Linear(self.features,num_classes)
    self.reconstruct_layer = nn.Linear(self.features,3072)
    self.loss = nn.CrossEntropyLoss()
    self.reconstruct = nn.MSELoss()

  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss1 = self.loss(logits, y)
    reconstruct_matrix = x
    for name, layer in self.model.named_children():
      if name=='fc':
        reconstruct_matrix = self.reconstruct_layer(reconstruct_matrix.reshape(reconstruct_matrix.shape[0],self.features))
        break
      else:
        reconstruct_matrix = layer(reconstruct_matrix)
    loss2 = self.reconstruct(reconstruct_matrix, x.reshape(x.shape[0],-1))
    total = self.w1*loss1+self.w2*loss2
    return total

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), weight_decay=0.0001, momentum=0.9, lr=0.1)

def transform_fn(img, mode='Train', resize=False):
  if resize==True:
    img = img.resize((224,224))
  tensor = transforms.ToTensor()(img)
  return tensor

def data_driven_transform_fn1(img, mode='Train', resize=False):
  if resize==True:
    img = img.resize((224,224))
  tensor = transforms.ToTensor()(img)
  tensor = transforms.RandomAdjustSharpness(sharpness_factor=2,p=0.5)(tensor)
  return tensor

def data_driven_transform_fn2(img, mode='Train', resize=False):
  if resize==True:
    img = img.resize((224,224))
  tensor = transforms.ToTensor()(img)
  tensor = transforms.RandomAutocontrast(p=0.5)(tensor)
  return tensor

def data_driven_transform_fn3(img, mode='Train', resize=False):
  if resize==True:
    img = img.resize((224,224))
  tensor = transforms.ToTensor()(img)
  tensor = transforms.RandomAdjustSharpness(sharpness_factor=2,p=0.5)(tensor)
  tensor = transforms.RandomAutocontrast(p=0.5)(tensor)
  return tensor

def model_return(ckpt_path, model_obj):
  ckpt = torch.load(ckpt_path)
  model_obj.load_state_dict(ckpt['state_dict'])
  return model_obj.eval().cuda()

def detect(test_loader, model_class):
  pred = []
  true = []
  for i,j in tqdm(test_loader):
    i,j = i.cuda(), j.cuda()
    prob = model_class(i)
    out = prob.max(1, keepdim=True)[1]
    pred.append(out.detach().cpu().item())
    true.append(j.detach().cpu().item())
  print(classification_report(true,pred))
  cm = confusion_matrix(true, pred)
  num_samples = np.sum(cm)
  num_samples_per_class = np.sum(cm, axis=1)
  tpr_per_class = np.diag(cm) / num_samples_per_class
  fpr_per_class = np.array(np.sum(cm, axis=0) - np.diag(cm),dtype='float64')
  fpr_per_class /= (num_samples - num_samples_per_class)
  ppv_per_class = np.diag(cm) / np.sum(cm, axis=0)
  npv_per_class = np.zeros_like(tpr_per_class)
  for i in range(len(npv_per_class)):
    idx = np.arange(len(npv_per_class)) != i
    tn = np.sum(cm[np.ix_(idx, idx)])
    fp = np.sum(cm[idx, i])
    npv_per_class[i] = tn / (tn + fp)
  macro_tpr = np.mean(tpr_per_class)
  macro_fpr = np.mean(fpr_per_class)
  macro_ppv = np.mean(ppv_per_class)
  macro_npv = np.mean(npv_per_class)
  micro_tpr = np.sum(np.diag(cm)) / num_samples
  micro_fpr = np.sum(fpr_per_class * num_samples_per_class) / num_samples
  micro_ppv = np.sum(np.diag(cm)) / np.sum(cm)
  micro_npv = np.sum(np.diag(cm)) / np.sum(cm)
  print("TPR_PER_CLASS = ",tpr_per_class)
  print("FPR_PER_CLASS = ",fpr_per_class)
  print("PPV_PER_CLASS = ",ppv_per_class)
  print("NPV_PER_CLASS = ",npv_per_class)
  print("MACRO_TPR = ",macro_tpr)
  print("MACRO_FPR = ",macro_fpr)
  print("MACRO_PPV = ",macro_ppv)
  print("MACRO_NPV = ",macro_npv)
  print("MICRO_TPR = ",micro_tpr)
  print("MICRO_FPR = ",micro_fpr)
  print("MICRO_PPV = ",micro_ppv)
  print("MICRO_NPV = ",micro_npv)

def calculate_tpr_fpr(cm, positive_label):
    tp = cm[positive_label, positive_label]
    fn = np.sum(cm[positive_label, :]) - tp
    fp = np.sum(cm[:, positive_label]) - tp
    tn = np.sum(cm) - tp - fn - fp
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return tpr, fpr

def Equalize_Odds_detect(model_class, protected_attr):
  dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_fn)
  if protected_attr=='pet_animal':
    pet_indices = [i for i in range(len(dataset)) if dataset.targets[i] ==2 or dataset.targets[i] == 3 or dataset.targets[i] == 5 or dataset.targets[i] == 7]
    non_pet_indices = [i for i in range(len(dataset)) if dataset.targets[i] == 4 or dataset.targets[i] == 6]
    protected_indices = pet_indices if protected_attr == 'pet_animal' else non_pet_indices
    non_protected_indices = non_pet_indices if protected_attr == 'pet_animal' else pet_indices
  protected_dataset = torch.utils.data.Subset(dataset, protected_indices)
  non_protected_dataset = torch.utils.data.Subset(dataset, non_protected_indices)
  protected_dataloader = torch.utils.data.DataLoader(protected_dataset, batch_size=1, shuffle=False)
  non_protected_dataloader = torch.utils.data.DataLoader(non_protected_dataset, batch_size=1, shuffle=False)
  conf_matrices = []
  tpr_protected = []
  fpr_protected = []
  tpr_non_protected = []
  fpr_non_protected = []
  t = []
  for dataloader in [protected_dataloader, non_protected_dataloader]:
    pred = []
    true = []
    correct = 0
    total = 0
    for images, labels in tqdm(dataloader):
      images, labels = images.cuda(), labels.cuda()
      outputs = model_class(images)
      _, predicted = torch.max(outputs.data, 1)
      pred.append(predicted.detach().cpu().item())
      true.append(labels.detach().cpu().item())   
      total+=1 
    print(classification_report(true, pred))
    conf_matrices.append(confusion_matrix(true, pred))
  for class_ind in range(10):
    tpr_protected_i, fpr_protected_i = calculate_tpr_fpr(conf_matrices[0], class_ind)
    tpr_non_protected_i, fpr_non_protected_i = calculate_tpr_fpr(conf_matrices[1], class_ind)
    tpr_protected.append(tpr_protected_i)
    fpr_protected.append(fpr_protected_i)
    tpr_non_protected.append(tpr_non_protected_i)
    fpr_non_protected.append(fpr_non_protected_i)
  tpr_protected = np.nan_to_num(np.array(tpr_protected),nan=0.0)
  tpr_non_protected = np.nan_to_num(np.array(tpr_non_protected),nan=0.0)
  fpr_protected = np.nan_to_num(np.array(fpr_protected),nan=0.0)
  fpr_non_protected = np.nan_to_num(np.array(fpr_non_protected),nan=0.0)
  delta_tpr = tpr_protected - tpr_non_protected
  delta_fpr = fpr_protected - fpr_non_protected
  avg_delta_tpr_fpr = np.mean([delta_tpr, delta_fpr], axis=0)
  thresholds = avg_delta_tpr_fpr / 2
  cm_protected_thresh = np.zeros_like(conf_matrices[0])
  cm_non_protected_thresh = np.zeros_like(conf_matrices[1])
  for i in range(10):
    for j in range(10):
      if i == j:
        cm_protected_thresh[i, j] = conf_matrices[0][i, j] if tpr_protected[i] > thresholds[i] else 0
        cm_non_protected_thresh[i, j] = conf_matrices[1][i, j] if tpr_non_protected[i] > thresholds[i] else 0
      else:
        cm_protected_thresh[i, j] = conf_matrices[0][i, j] if fpr_protected[i] < thresholds[i] else 0
        cm_non_protected_thresh[i, j] = conf_matrices[1][i, j] if fpr_non_protected[i] < thresholds[i] else 0
  print("Confusion Matrix before Equality Odds Difference for Protected Groups",conf_matrices[0])
  print("Confusion Matrix before Equality Odds Difference for Non-Protected Groups",conf_matrices[1])
  print("Confusion Matrix after Equality Odds Difference for Protected Groups",cm_protected_thresh)
  print("Confusion Matrix after Equality Odds Difference for Non-Protected Groups",cm_non_protected_thresh)
  print(" ")
  
  num_samples = np.sum(conf_matrices[0])
  num_samples_per_class = np.sum(conf_matrices[0], axis=1)
  tpr_per_class = np.diag(conf_matrices[0]) / num_samples_per_class
  fpr_per_class = np.array(np.sum(conf_matrices[0], axis=0) - np.diag(conf_matrices[0]),dtype='float64')
  fpr_per_class /= (num_samples - num_samples_per_class)
  ppv_per_class = np.diag(conf_matrices[0]) / np.sum(conf_matrices[0], axis=0)
  npv_per_class = np.zeros_like(tpr_per_class)
  for i in range(len(npv_per_class)):
    idx = np.arange(len(npv_per_class)) != i
    tn = np.sum(conf_matrices[0][np.ix_(idx, idx)])
    fp = np.sum(conf_matrices[0][idx, i])
    npv_per_class[i] = tn / (tn + fp)
  macro_tpr = np.mean(tpr_per_class)
  macro_fpr = np.mean(fpr_per_class)
  macro_ppv = np.mean(ppv_per_class)
  macro_npv = np.mean(npv_per_class)
  micro_tpr = np.sum(np.diag(conf_matrices[0])) / num_samples
  micro_fpr = np.sum(fpr_per_class * num_samples_per_class) / num_samples
  micro_ppv = np.sum(np.diag(conf_matrices[0])) / np.sum(conf_matrices[0])
  micro_npv = np.sum(np.diag(conf_matrices[0])) / np.sum(conf_matrices[0])
  print("TPR_PER_CLASS before Equality Odds Difference for Protected Groups = ",tpr_per_class)
  print("FPR_PER_CLASS before Equality Odds Difference for Protected Groups = ",fpr_per_class)
  print("PPV_PER_CLASS before Equality Odds Difference for Protected Groups = ",ppv_per_class)
  print("NPV_PER_CLASS before Equality Odds Difference for Protected Groups = ",npv_per_class)
  print("MACRO_TPR before Equality Odds Difference for Protected Groups = ",macro_tpr)
  print("MACRO_FPR before Equality Odds Difference for Protected Groups = ",macro_fpr)
  print("MACRO_PPV before Equality Odds Difference for Protected Groups = ",macro_ppv)
  print("MACRO_NPV before Equality Odds Difference for Protected Groups = ",macro_npv)
  print("MICRO_TPR before Equality Odds Difference for Protected Groups = ",micro_tpr)
  print("MICRO_FPR before Equality Odds Difference for Protected Groups = ",micro_fpr)
  print("MICRO_PPV before Equality Odds Difference for Protected Groups = ",micro_ppv)
  print("MICRO_NPV before Equality Odds Difference for Protected Groups = ",micro_npv)
  print(" ")
  num_samples = np.sum(conf_matrices[1])
  num_samples_per_class = np.sum(conf_matrices[1], axis=1)
  tpr_per_class = np.diag(conf_matrices[1]) / num_samples_per_class
  fpr_per_class = np.array(np.sum(conf_matrices[1], axis=0) - np.diag(conf_matrices[1]),dtype='float64')
  fpr_per_class /= (num_samples - num_samples_per_class)
  ppv_per_class = np.diag(conf_matrices[1]) / np.sum(conf_matrices[1], axis=0)
  npv_per_class = np.zeros_like(tpr_per_class)
  for i in range(len(npv_per_class)):
    idx = np.arange(len(npv_per_class)) != i
    tn = np.sum(conf_matrices[1][np.ix_(idx, idx)])
    fp = np.sum(conf_matrices[1][idx, i])
    npv_per_class[i] = tn / (tn + fp)
  macro_tpr = np.mean(tpr_per_class)
  macro_fpr = np.mean(fpr_per_class)
  macro_ppv = np.mean(ppv_per_class)
  macro_npv = np.mean(npv_per_class)
  micro_tpr = np.sum(np.diag(conf_matrices[1])) / num_samples
  micro_fpr = np.sum(fpr_per_class * num_samples_per_class) / num_samples
  micro_ppv = np.sum(np.diag(conf_matrices[1])) / np.sum(conf_matrices[1])
  micro_npv = np.sum(np.diag(conf_matrices[1])) / np.sum(conf_matrices[1])
  print("TPR_PER_CLASS before Equality Odds Difference for Non-Protected Groups = ",tpr_per_class)
  print("FPR_PER_CLASS before Equality Odds Difference for Non-Protected Groups = ",fpr_per_class)
  print("PPV_PER_CLASS before Equality Odds Difference for Non-Protected Groups = ",ppv_per_class)
  print("NPV_PER_CLASS before Equality Odds Difference for Non-Protected Groups = ",npv_per_class)
  print("MACRO_TPR before Equality Odds Difference for Non-Protected Groups = ",macro_tpr)
  print("MACRO_FPR before Equality Odds Difference for Non-Protected Groups = ",macro_fpr)
  print("MACRO_PPV before Equality Odds Difference for Non-Protected Groups = ",macro_ppv)
  print("MACRO_NPV before Equality Odds Difference for Non-Protected Groups = ",macro_npv)
  print("MICRO_TPR before Equality Odds Difference for Non-Protected Groups = ",micro_tpr)
  print("MICRO_FPR before Equality Odds Difference for Non-Protected Groups = ",micro_fpr)
  print("MICRO_PPV before Equality Odds Difference for Non-Protected Groups = ",micro_ppv)
  print("MICRO_NPV before Equality Odds Difference for Non-Protected Groups = ",micro_npv)
  print(" ")
  num_samples = np.sum(cm_protected_thresh)
  num_samples_per_class = np.sum(cm_protected_thresh, axis=1)
  tpr_per_class = np.diag(cm_protected_thresh) / num_samples_per_class
  fpr_per_class = np.array(np.sum(cm_protected_thresh, axis=0) - np.diag(cm_protected_thresh),dtype='float64')
  fpr_per_class /= (num_samples - num_samples_per_class)
  ppv_per_class = np.diag(cm_protected_thresh) / np.sum(cm_protected_thresh, axis=0)
  npv_per_class = np.zeros_like(tpr_per_class)
  for i in range(len(npv_per_class)):
    idx = np.arange(len(npv_per_class)) != i
    tn = np.sum(cm_protected_thresh[np.ix_(idx, idx)])
    fp = np.sum(cm_protected_thresh[idx, i])
    npv_per_class[i] = tn / (tn + fp)
  macro_tpr = np.mean(tpr_per_class)
  macro_fpr = np.mean(fpr_per_class)
  macro_ppv = np.mean(ppv_per_class)
  macro_npv = np.mean(npv_per_class)
  micro_tpr = np.sum(np.diag(cm_protected_thresh)) / num_samples
  micro_fpr = np.sum(fpr_per_class * num_samples_per_class) / num_samples
  micro_ppv = np.sum(np.diag(cm_protected_thresh)) / np.sum(cm_protected_thresh)
  micro_npv = np.sum(np.diag(cm_protected_thresh)) / np.sum(cm_protected_thresh)
  print("TPR_PER_CLASS after Equality Odds Difference for Protected Groups = ",tpr_per_class)
  print("FPR_PER_CLASS after Equality Odds Difference for Protected Groups = ",fpr_per_class)
  print("PPV_PER_CLASS after Equality Odds Difference for Protected Groups = ",ppv_per_class)
  print("NPV_PER_CLASS after Equality Odds Difference for Protected Groups = ",npv_per_class)
  print("MACRO_TPR after Equality Odds Difference for Protected Groups = ",macro_tpr)
  print("MACRO_FPR after Equality Odds Difference for Protected Groups = ",macro_fpr)
  print("MACRO_PPV after Equality Odds Difference for Protected Groups = ",macro_ppv)
  print("MACRO_NPV after Equality Odds Difference for Protected Groups = ",macro_npv)
  print("MICRO_TPR after Equality Odds Difference for Protected Groups = ",micro_tpr)
  print("MICRO_FPR after Equality Odds Difference for Protected Groups = ",micro_fpr)
  print("MICRO_PPV after Equality Odds Difference for Protected Groups = ",micro_ppv)
  print("MICRO_NPV after Equality Odds Difference for Protected Groups = ",micro_npv)
  print(" ")
  num_samples = np.sum(cm_non_protected_thresh)
  num_samples_per_class = np.sum(cm_non_protected_thresh, axis=1)
  tpr_per_class = np.diag(cm_non_protected_thresh) / num_samples_per_class
  fpr_per_class = np.array(np.sum(cm_non_protected_thresh, axis=0) - np.diag(cm_non_protected_thresh),dtype='float64')
  fpr_per_class /= (num_samples - num_samples_per_class)
  ppv_per_class = np.diag(cm_non_protected_thresh) / np.sum(cm_non_protected_thresh, axis=0)
  npv_per_class = np.zeros_like(tpr_per_class)
  for i in range(len(npv_per_class)):
    idx = np.arange(len(npv_per_class)) != i
    tn = np.sum(cm_non_protected_thresh[np.ix_(idx, idx)])
    fp = np.sum(cm_non_protected_thresh[idx, i])
    npv_per_class[i] = tn / (tn + fp)
  macro_tpr = np.mean(tpr_per_class)
  macro_fpr = np.mean(fpr_per_class)
  macro_ppv = np.mean(ppv_per_class)
  macro_npv = np.mean(npv_per_class)
  micro_tpr = np.sum(np.diag(cm_non_protected_thresh)) / num_samples
  micro_fpr = np.sum(fpr_per_class * num_samples_per_class) / num_samples
  micro_ppv = np.sum(np.diag(cm_non_protected_thresh)) / np.sum(cm_non_protected_thresh)
  micro_npv = np.sum(np.diag(cm_non_protected_thresh)) / np.sum(cm_non_protected_thresh)
  print("TPR_PER_CLASS after Equality Odds Difference for Non-Protected Groups = ",tpr_per_class)
  print("FPR_PER_CLASS after Equality Odds Difference for Non-Protected Groups = ",fpr_per_class)
  print("PPV_PER_CLASS after Equality Odds Difference for Non-Protected Groups = ",ppv_per_class)
  print("NPV_PER_CLASS after Equality Odds Difference for Non-Protected Groups = ",npv_per_class)
  print("MACRO_TPR after Equality Odds Difference for Non-Protected Groups = ",macro_tpr)
  print("MACRO_FPR after Equality Odds Difference for Non-Protected Groups = ",macro_fpr)
  print("MACRO_PPV after Equality Odds Difference for Non-Protected Groups = ",macro_ppv)
  print("MACRO_NPV after Equality Odds Difference for Non-Protected Groups = ",macro_npv)
  print("MICRO_TPR after Equality Odds Difference for Non-Protected Groups = ",micro_tpr)
  print("MICRO_FPR after Equality Odds Difference for Non-Protected Groups = ",micro_fpr)
  print("MICRO_PPV after Equality Odds Difference for Non-Protected Groups = ",micro_ppv)
  print("MICRO_NPV after Equality Odds Difference for Non-Protected Groups = ",micro_npv)

#CIFAR-10 (Reproducing the Results)

In [ ]:
train_CIFAR10 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=True, download=True, transform=transform_fn),batch_size=256, shuffle=True, pin_memory=True, num_workers=2)
test_CIFAR10 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=False, download=True, transform=transform_fn),batch_size=1, shuffle=True)

100%|██████████| 170498071/170498071 [00:13<00:00, 13038666.58it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


In [ ]:
Model_50_32x4d = ResNext(model_variant='50_32x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR10)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 23.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
92.002    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='50_32x4d',num_classes=10))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:12<00:00, 138.24it/s]


              precision    recall  f1-score   support

           0       0.70      0.76      0.73      1000
           1       0.86      0.75      0.80      1000
           2       0.60      0.56      0.58      1000
           3       0.41      0.64      0.50      1000
           4       0.69      0.54      0.60      1000
           5       0.59      0.53      0.56      1000
           6       0.78      0.68      0.73      1000
           7       0.76      0.71      0.73      1000
           8       0.74      0.84      0.79      1000
           9       0.79      0.71      0.75      1000

    accuracy                           0.67     10000
   macro avg       0.69      0.67      0.68     10000
weighted avg       0.69      0.67      0.68     10000

TPR_PER_CLASS =  [0.757 0.748 0.563 0.638 0.537 0.529 0.684 0.709 0.845 0.71 ]
FPR_PER_CLASS =  [0.03666667 0.013      0.04122222 0.10344444 0.02677778 0.04155556
 0.02188889 0.02488889 0.03344444 0.02155556]
PPV_PER_CLASS =  [0.69641214 0.8

In [ ]:
Model_101_32x8d = ResNext(model_variant='101_32x8d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/101_32x8d', benchmark=True)
trainer.fit(Model_101_32x8d, train_CIFAR10)
del trainer, Model_101_32x8d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 86.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
86.8 M    Trainable params
0         Non-trainable params
86.8 M    Total params
347.051   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_32x8d = model_return('/content/drive/MyDrive/DAI_Assignment_2/101_32x8d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_32x8d',num_classes=10))
detect(test_CIFAR10, Model_101_32x8d)
del Model_101_32x8d

100%|██████████| 10000/10000 [03:09<00:00, 52.67it/s]


              precision    recall  f1-score   support

           0       0.66      0.74      0.70      1000
           1       0.84      0.65      0.73      1000
           2       0.52      0.59      0.55      1000
           3       0.47      0.45      0.46      1000
           4       0.58      0.60      0.59      1000
           5       0.55      0.60      0.58      1000
           6       0.75      0.71      0.73      1000
           7       0.72      0.70      0.71      1000
           8       0.87      0.67      0.76      1000
           9       0.64      0.76      0.70      1000

    accuracy                           0.65     10000
   macro avg       0.66      0.65      0.65     10000
weighted avg       0.66      0.65      0.65     10000

TPR_PER_CLASS =  [0.744 0.654 0.591 0.446 0.605 0.602 0.71  0.699 0.673 0.764]
FPR_PER_CLASS =  [0.04344444 0.01411111 0.061      0.05477778 0.04877778 0.05366667
 0.02611111 0.03011111 0.01144444 0.04677778]
PPV_PER_CLASS =  [0.65550661 0.8

In [ ]:
Model_101_64x4d = ResNext(model_variant='101_64x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/101_64x4d', benchmark=True)
trainer.fit(Model_101_64x4d, train_CIFAR10)
del trainer, Model_101_64x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 81.4 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
81.4 M    Trainable params
0         Non-trainable params
81.4 M    Total params
325.707   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_64x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/101_64x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_64x4d',num_classes=10))
detect(test_CIFAR10, Model_101_64x4d)
del Model_101_64x4d

100%|██████████| 10000/10000 [02:21<00:00, 70.58it/s]

              precision    recall  f1-score   support

           0       0.68      0.70      0.69      1000
           1       0.69      0.77      0.73      1000
           2       0.51      0.58      0.54      1000
           3       0.44      0.45      0.44      1000
           4       0.66      0.48      0.56      1000
           5       0.55      0.53      0.54      1000
           6       0.70      0.71      0.71      1000
           7       0.76      0.64      0.69      1000
           8       0.75      0.75      0.75      1000
           9       0.64      0.74      0.69      1000

    accuracy                           0.64     10000
   macro avg       0.64      0.64      0.63     10000
weighted avg       0.64      0.64      0.63     10000

TPR_PER_CLASS =  [0.704 0.771 0.576 0.446 0.483 0.534 0.713 0.642 0.751 0.737]
FPR_PER_CLASS =  [0.03622222 0.03833333 0.06122222 0.06311111 0.02788889 0.04877778
 0.03322222 0.02288889 0.02777778 0.04533333]
PPV_PER_CLASS =  [0.68349515 0.6

# CIFAR-10 with Equalized-Odds, a Post-Processing based Bias Mitigation Technique

In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='50_32x4d',num_classes=10))
Equalize_Odds_detect(Model_50_32x4d, 'pet_animal')
del Model_50_32x4d

100%|██████████| 170498071/170498071 [00:13<00:00, 12596511.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


100%|██████████| 4000/4000 [00:30<00:00, 131.62it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.83      0.56      0.67      1000
           3       0.54      0.64      0.58      1000
           4       0.00      0.00      0.00         0
           5       0.69      0.53      0.60      1000
           6       0.00      0.00      0.00         0
           7       0.87      0.71      0.78      1000
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.61      4000
   macro avg       0.29      0.24      0.26      4000
weighted avg       0.73      0.61      0.66      4000



100%|██████████| 2000/2000 [00:14<00:00, 139.13it/s]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.91      0.54      0.68      1000
           5       0.00      0.00      0.00         0
           6       0.95      0.68      0.80      1000
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.61      2000
   macro avg       0.19      0.12      0.15      2000
weighted avg       0.93      0.61      0.74      2000

Confusion Matrix before Equality Odds Difference for Protected Groups [[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 78   4 563 136  57  43  54  32  25   8]
 [ 34   5  49 638  40 134  40  33  12  15]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-24-c359e30f8ad9>:158: RuntimeWarning: invalid value encountered in long_sca

In [ ]:
Model_101_32x8d = model_return('/content/drive/MyDrive/DAI_Assignment_2/101_32x8d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_32x8d',num_classes=10))
Equalize_Odds_detect(Model_101_32x8d, 'pet_animal')
del Model_101_32x8d

Files already downloaded and verified


100%|██████████| 4000/4000 [01:15<00:00, 52.83it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.72      0.59      0.65      1000
           3       0.64      0.45      0.52      1000
           4       0.00      0.00      0.00         0
           5       0.64      0.60      0.62      1000
           6       0.00      0.00      0.00         0
           7       0.83      0.70      0.76      1000
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.58      4000
   macro avg       0.28      0.23      0.26      4000
weighted avg       0.71      0.58      0.64      4000



100%|██████████| 2000/2000 [00:35<00:00, 56.92it/s]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.93      0.60      0.73      1000
           5       0.00      0.00      0.00         0
           6       0.94      0.71      0.81      1000
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.66      2000
   macro avg       0.19      0.13      0.15      2000
weighted avg       0.93      0.66      0.77      2000

Confusion Matrix before Equality Odds Difference for Protected Groups [[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 56   4 591  75  98  61  56  40   6  13]
 [ 32   4  98 446  80 210  54  43   3  30]


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-24-c359e30f8ad9>:158: RuntimeWarning: invalid value encountered in long_sca

In [ ]:
Model_101_64x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/101_64x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_64x4d',num_classes=10))
Equalize_Odds_detect(Model_101_64x4d, 'pet_animal')
del Model_101_64x4d

Files already downloaded and verified


100%|██████████| 4000/4000 [00:56<00:00, 70.81it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.70      0.58      0.63      1000
           3       0.57      0.45      0.50      1000
           4       0.00      0.00      0.00         0
           5       0.62      0.53      0.58      1000
           6       0.00      0.00      0.00         0
           7       0.85      0.64      0.73      1000
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.55      4000
   macro avg       0.27      0.22      0.24      4000
weighted avg       0.69      0.55      0.61      4000



100%|██████████| 2000/2000 [00:27<00:00, 71.67it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-24-c359e30f8ad9>:158: Ru

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.93      0.48      0.64      1000
           5       0.00      0.00      0.00         0
           6       0.89      0.71      0.79      1000
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.60      2000
   macro avg       0.18      0.12      0.14      2000
weighted avg       0.91      0.60      0.71      2000

Confusion Matrix before Equality Odds Difference for Protected Groups [[  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [ 55  12 576  76  59  53  67  42  24  36]
 [ 28  20 102 446  44 195  69  29  28  39]

<ipython-input-24-c359e30f8ad9>:325: RuntimeWarning: invalid value encountered in true_divide
  tpr_per_class = np.diag(cm_non_protected_thresh) / num_samples_per_class
<ipython-input-24-c359e30f8ad9>:328: RuntimeWarning: invalid value encountered in true_divide
  ppv_per_class = np.diag(cm_non_protected_thresh) / np.sum(cm_non_protected_thresh, axis=0)


#CIFAR-10 with Data Preprocessing (3 types)

## Random Sharpness

In [ ]:
train_CIFAR101 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=True, download=True, transform=data_driven_transform_fn1),batch_size=256, shuffle=True, pin_memory=True, num_workers=2)

Files already downloaded and verified


In [ ]:
Model_50_32x4d = ResNext(model_variant='50_32x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR101)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 23.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
92.002    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data_50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='50_32x4d',num_classes=10))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:13<00:00, 136.48it/s]


              precision    recall  f1-score   support

           0       0.76      0.73      0.75      1000
           1       0.84      0.76      0.80      1000
           2       0.59      0.61      0.60      1000
           3       0.48      0.54      0.51      1000
           4       0.62      0.61      0.62      1000
           5       0.61      0.56      0.59      1000
           6       0.69      0.79      0.74      1000
           7       0.82      0.66      0.73      1000
           8       0.80      0.81      0.81      1000
           9       0.73      0.79      0.75      1000

    accuracy                           0.69     10000
   macro avg       0.69      0.69      0.69     10000
weighted avg       0.69      0.69      0.69     10000

TPR_PER_CLASS =  [0.73  0.761 0.609 0.538 0.615 0.565 0.792 0.664 0.814 0.785]
FPR_PER_CLASS =  [0.02511111 0.01555556 0.04733333 0.06555556 0.04188889 0.04055556
 0.03922222 0.01655556 0.02277778 0.03288889]
PPV_PER_CLASS =  [0.76359833 0.8

In [ ]:
Model_101_32x8d = ResNext(model_variant='101_32x8d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data_101_32x8d', benchmark=True)
trainer.fit(Model_101_32x8d, train_CIFAR101)
del trainer, Model_101_32x8d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 86.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
86.8 M    Trainable params
0         Non-trainable params
86.8 M    Total params
347.051   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_32x8d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data_101_32x8d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_32x8d',num_classes=10))
detect(test_CIFAR10, Model_101_32x8d)
del Model_101_32x8d

100%|██████████| 10000/10000 [03:01<00:00, 55.09it/s]

              precision    recall  f1-score   support

           0       0.72      0.71      0.72      1000
           1       0.83      0.72      0.77      1000
           2       0.49      0.58      0.53      1000
           3       0.46      0.45      0.45      1000
           4       0.57      0.57      0.57      1000
           5       0.49      0.58      0.53      1000
           6       0.71      0.70      0.71      1000
           7       0.75      0.65      0.69      1000
           8       0.76      0.75      0.75      1000
           9       0.69      0.67      0.68      1000

    accuracy                           0.64     10000
   macro avg       0.65      0.64      0.64     10000
weighted avg       0.65      0.64      0.64     10000

TPR_PER_CLASS =  [0.713 0.725 0.576 0.446 0.572 0.583 0.7   0.648 0.747 0.673]
FPR_PER_CLASS =  [0.03033333 0.01688889 0.06622222 0.05822222 0.04888889 0.06655556
 0.03122222 0.02444444 0.02577778 0.03333333]
PPV_PER_CLASS =  [0.72312373 0.8

In [ ]:
Model_101_64x4d = ResNext(model_variant='101_64x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data_101_64x4d', benchmark=True)
trainer.fit(Model_101_64x4d, train_CIFAR101)
del trainer, Model_101_64x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 81.4 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
81.4 M    Trainable params
0         Non-trainable params
81.4 M    Total params
325.707   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_64x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data_101_64x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_64x4d',num_classes=10))
detect(test_CIFAR10, Model_101_64x4d)
del Model_101_64x4d

100%|██████████| 10000/10000 [02:21<00:00, 70.63it/s]

              precision    recall  f1-score   support

           0       0.70      0.73      0.71      1000
           1       0.84      0.72      0.78      1000
           2       0.58      0.53      0.55      1000
           3       0.43      0.55      0.48      1000
           4       0.60      0.61      0.61      1000
           5       0.54      0.58      0.56      1000
           6       0.77      0.67      0.71      1000
           7       0.75      0.66      0.70      1000
           8       0.71      0.80      0.75      1000
           9       0.76      0.74      0.75      1000

    accuracy                           0.66     10000
   macro avg       0.67      0.66      0.66     10000
weighted avg       0.67      0.66      0.66     10000

TPR_PER_CLASS =  [0.729 0.723 0.525 0.549 0.607 0.58  0.666 0.659 0.798 0.738]
FPR_PER_CLASS =  [0.03488889 0.01477778 0.042      0.08011111 0.04433333 0.05555556
 0.02211111 0.024      0.03644444 0.02644444]
PPV_PER_CLASS =  [0.69894535 0.8

## Random AutoContrast

In [ ]:
train_CIFAR102 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=True, download=True, transform=data_driven_transform_fn2),batch_size=256, shuffle=True, pin_memory=True, num_workers=2)

Files already downloaded and verified


In [ ]:
Model_50_32x4d = ResNext(model_variant='50_32x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data2_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR102)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 23.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
92.002    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data2_50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='50_32x4d',num_classes=10))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:12<00:00, 137.15it/s]


              precision    recall  f1-score   support

           0       0.80      0.67      0.73      1000
           1       0.86      0.75      0.80      1000
           2       0.54      0.60      0.57      1000
           3       0.53      0.45      0.49      1000
           4       0.62      0.61      0.61      1000
           5       0.62      0.52      0.56      1000
           6       0.58      0.88      0.70      1000
           7       0.74      0.75      0.74      1000
           8       0.82      0.81      0.81      1000
           9       0.76      0.78      0.77      1000

    accuracy                           0.68     10000
   macro avg       0.69      0.68      0.68     10000
weighted avg       0.69      0.68      0.68     10000

TPR_PER_CLASS =  [0.67  0.75  0.596 0.445 0.609 0.517 0.881 0.747 0.807 0.783]
FPR_PER_CLASS =  [0.01888889 0.01388889 0.05555556 0.04311111 0.04211111 0.03544444
 0.07022222 0.029      0.01933333 0.02744444]
PPV_PER_CLASS =  [0.79761905 0.8

In [ ]:
Model_101_32x8d = ResNext(model_variant='101_32x8d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data2_101_32x8d', benchmark=True)
trainer.fit(Model_101_32x8d, train_CIFAR102)
del trainer, Model_101_32x8d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 86.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
86.8 M    Trainable params
0         Non-trainable params
86.8 M    Total params
347.051   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_32x8d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data2_101_32x8d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_32x8d',num_classes=10))
detect(test_CIFAR10, Model_101_32x8d)
del Model_101_32x8d

100%|██████████| 10000/10000 [03:02<00:00, 54.87it/s]


              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1000
           1       0.82      0.73      0.78      1000
           2       0.51      0.62      0.56      1000
           3       0.48      0.47      0.47      1000
           4       0.52      0.65      0.58      1000
           5       0.71      0.43      0.53      1000
           6       0.72      0.70      0.71      1000
           7       0.78      0.61      0.69      1000
           8       0.77      0.80      0.78      1000
           9       0.73      0.74      0.74      1000

    accuracy                           0.66     10000
   macro avg       0.67      0.66      0.65     10000
weighted avg       0.67      0.66      0.65     10000

TPR_PER_CLASS =  [0.795 0.735 0.616 0.469 0.653 0.427 0.701 0.612 0.797 0.745]
FPR_PER_CLASS =  [0.05044444 0.01733333 0.06688889 0.05677778 0.06566667 0.01977778
 0.02966667 0.01922222 0.02677778 0.03077778]
PPV_PER_CLASS =  [0.63650921 0.8

In [ ]:
Model_101_64x4d = ResNext(model_variant='101_64x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data2_101_64x4d', benchmark=True)
trainer.fit(Model_101_64x4d, train_CIFAR102)
del trainer, Model_101_64x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 81.4 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
81.4 M    Trainable params
0         Non-trainable params
81.4 M    Total params
325.707   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_64x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data2_101_64x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_64x4d',num_classes=10))
detect(test_CIFAR10, Model_101_64x4d)
del Model_101_64x4d

100%|██████████| 10000/10000 [02:19<00:00, 71.43it/s]

              precision    recall  f1-score   support

           0       0.57      0.82      0.67      1000
           1       0.76      0.79      0.78      1000
           2       0.61      0.41      0.49      1000
           3       0.51      0.35      0.42      1000
           4       0.53      0.61      0.57      1000
           5       0.61      0.53      0.57      1000
           6       0.59      0.80      0.68      1000
           7       0.75      0.69      0.72      1000
           8       0.78      0.72      0.75      1000
           9       0.75      0.70      0.73      1000

    accuracy                           0.64     10000
   macro avg       0.65      0.64      0.64     10000
weighted avg       0.65      0.64      0.64     10000

TPR_PER_CLASS =  [0.822 0.787 0.414 0.351 0.614 0.53  0.801 0.693 0.725 0.702]
FPR_PER_CLASS =  [0.06833333 0.02688889 0.02955556 0.03711111 0.06166667 0.03811111
 0.06066667 0.02522222 0.02233333 0.02577778]
PPV_PER_CLASS =  [0.57202505 0.7

## Both

In [ ]:
train_CIFAR103 = torch.utils.data.DataLoader(dataset=datasets.CIFAR10('../data', train=True, download=True, transform=data_driven_transform_fn3),batch_size=256, shuffle=True, pin_memory=True, num_workers=2)

Files already downloaded and verified


In [ ]:
Model_50_32x4d = ResNext(model_variant='50_32x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data3_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR103)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 23.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
23.0 M    Trainable params
0         Non-trainable params
23.0 M    Total params
92.002    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data3_50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='50_32x4d',num_classes=10))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:14<00:00, 134.31it/s]


              precision    recall  f1-score   support

           0       0.71      0.75      0.73      1000
           1       0.78      0.81      0.79      1000
           2       0.52      0.63      0.57      1000
           3       0.47      0.46      0.47      1000
           4       0.61      0.65      0.63      1000
           5       0.61      0.54      0.57      1000
           6       0.71      0.77      0.74      1000
           7       0.81      0.68      0.74      1000
           8       0.78      0.81      0.79      1000
           9       0.80      0.65      0.72      1000

    accuracy                           0.67     10000
   macro avg       0.68      0.67      0.67     10000
weighted avg       0.68      0.67      0.67     10000

TPR_PER_CLASS =  [0.748 0.805 0.63  0.46  0.645 0.541 0.772 0.682 0.807 0.653]
FPR_PER_CLASS =  [0.03466667 0.02466667 0.06344444 0.057      0.04555556 0.039
 0.03555556 0.01833333 0.02544444 0.01822222]
PPV_PER_CLASS =  [0.70566038 0.783836

In [ ]:
Model_101_32x8d = ResNext(model_variant='101_32x8d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data3_101_32x8d', benchmark=True)
trainer.fit(Model_101_32x8d, train_CIFAR103)
del trainer, Model_101_32x8d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 86.8 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
86.8 M    Trainable params
0         Non-trainable params
86.8 M    Total params
347.051   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_32x8d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data3_101_32x8d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_32x8d',num_classes=10))
detect(test_CIFAR10, Model_101_32x8d)
del Model_101_32x8d

100%|██████████| 10000/10000 [03:03<00:00, 54.59it/s]

              precision    recall  f1-score   support

           0       0.63      0.77      0.69      1000
           1       0.87      0.70      0.78      1000
           2       0.56      0.65      0.60      1000
           3       0.48      0.51      0.49      1000
           4       0.60      0.59      0.59      1000
           5       0.60      0.53      0.56      1000
           6       0.77      0.70      0.73      1000
           7       0.75      0.68      0.71      1000
           8       0.74      0.78      0.76      1000
           9       0.74      0.73      0.73      1000

    accuracy                           0.66     10000
   macro avg       0.67      0.66      0.67     10000
weighted avg       0.67      0.66      0.67     10000

TPR_PER_CLASS =  [0.765 0.705 0.649 0.505 0.588 0.531 0.704 0.68  0.78  0.731]
FPR_PER_CLASS =  [0.05055556 0.01177778 0.05744444 0.06111111 0.04366667 0.04
 0.02377778 0.025      0.03122222 0.029     ]
PPV_PER_CLASS =  [0.62704918 0.8692971

In [ ]:
Model_101_64x4d = ResNext(model_variant='101_64x4d',num_classes=10).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/data3_101_64x4d', benchmark=True)
trainer.fit(Model_101_64x4d, train_CIFAR103)
del trainer, Model_101_64x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 81.4 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
81.4 M    Trainable params
0         Non-trainable params
81.4 M    Total params
325.707   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_101_64x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/data3_101_64x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext(model_variant='101_64x4d',num_classes=10))
detect(test_CIFAR10, Model_101_64x4d)
del Model_101_64x4d

100%|██████████| 10000/10000 [02:19<00:00, 71.47it/s]


              precision    recall  f1-score   support

           0       0.80      0.62      0.70      1000
           1       0.77      0.81      0.79      1000
           2       0.48      0.62      0.54      1000
           3       0.49      0.40      0.44      1000
           4       0.61      0.55      0.58      1000
           5       0.50      0.67      0.57      1000
           6       0.75      0.68      0.71      1000
           7       0.77      0.69      0.72      1000
           8       0.75      0.81      0.78      1000
           9       0.76      0.71      0.73      1000

    accuracy                           0.66     10000
   macro avg       0.67      0.66      0.66     10000
weighted avg       0.67      0.66      0.66     10000

TPR_PER_CLASS =  [0.625 0.815 0.623 0.404 0.549 0.668 0.681 0.687 0.805 0.708]
FPR_PER_CLASS =  [0.01755556 0.02677778 0.07411111 0.046      0.03833333 0.07488889
 0.02544444 0.02344444 0.02988889 0.02522222]
PPV_PER_CLASS =  [0.79821201 0.7

#CIFAR-10 with Multi-Tasking (3 types)

## Equal Weights

In [ ]:
Model_50_32x4d = ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.5,w2=0.5).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/model_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR10)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params
-------------------------------------------------------
0 | model             | ResNet           | 23.0 M
1 | reconstruct_layer | Linear           | 6.3 M 
2 | loss              | CrossEntropyLoss | 0     
3 | reconstruct       | MSELoss          | 0     
-------------------------------------------------------
29.3 M    Trainable params
0         Non-trainable params
29.3 M    Total params
117.180   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/model_50_32x4d/lightning_logs/version_9/checkpoints/epoch=99-step=19600.ckpt', ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.5,w2=0.5))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:13<00:00, 136.36it/s]


              precision    recall  f1-score   support

           0       0.75      0.69      0.72      1000
           1       0.81      0.74      0.77      1000
           2       0.66      0.49      0.57      1000
           3       0.50      0.44      0.47      1000
           4       0.58      0.64      0.61      1000
           5       0.54      0.59      0.57      1000
           6       0.70      0.74      0.72      1000
           7       0.69      0.71      0.70      1000
           8       0.76      0.83      0.79      1000
           9       0.68      0.78      0.73      1000

    accuracy                           0.67     10000
   macro avg       0.67      0.67      0.67     10000
weighted avg       0.67      0.67      0.67     10000

TPR_PER_CLASS =  [0.694 0.744 0.495 0.438 0.644 0.594 0.739 0.715 0.83  0.783]
FPR_PER_CLASS =  [0.02577778 0.01977778 0.02855556 0.04855556 0.05166667 0.05566667
 0.03477778 0.035      0.02922222 0.04033333]
PPV_PER_CLASS =  [0.74946004 0.8

## MSE more than Cross Entropy

In [ ]:
Model_50_32x4d = ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.3,w2=0.7).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/model2_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR10)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params
-------------------------------------------------------
0 | model             | ResNet           | 23.0 M
1 | reconstruct_layer | Linear           | 6.3 M 
2 | loss              | CrossEntropyLoss | 0     
3 | reconstruct       | MSELoss          | 0     
-------------------------------------------------------
29.3 M    Trainable params
0         Non-trainable params
29.3 M    Total params
117.180   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/model2_50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.3,w2=0.7))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:14<00:00, 133.98it/s]

              precision    recall  f1-score   support

           0       0.74      0.74      0.74      1000
           1       0.83      0.79      0.81      1000
           2       0.68      0.54      0.60      1000
           3       0.50      0.48      0.49      1000
           4       0.65      0.65      0.65      1000
           5       0.59      0.65      0.62      1000
           6       0.79      0.73      0.76      1000
           7       0.76      0.77      0.76      1000
           8       0.78      0.85      0.81      1000
           9       0.71      0.83      0.77      1000

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000

TPR_PER_CLASS =  [0.744 0.791 0.537 0.477 0.645 0.653 0.734 0.765 0.847 0.832]
FPR_PER_CLASS =  [0.02833333 0.01766667 0.02844444 0.05366667 0.039      0.05122222
 0.02133333 0.02622222 0.02666667 0.038     ]
PPV_PER_CLASS =  [0.74474474 0.8

## Cross Entropy more than MSE

In [ ]:
Model_50_32x4d = ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.7,w2=0.3).train().cuda()
trainer = pl.Trainer(accelerator='gpu', max_epochs=100, default_root_dir='/content/drive/MyDrive/DAI_Assignment_2/model3_50_32x4d', benchmark=True)
trainer.fit(Model_50_32x4d, train_CIFAR10)
del trainer, Model_50_32x4d

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type             | Params
-------------------------------------------------------
0 | model             | ResNet           | 23.0 M
1 | reconstruct_layer | Linear           | 6.3 M 
2 | loss              | CrossEntropyLoss | 0     
3 | reconstruct       | MSELoss          | 0     
-------------------------------------------------------
29.3 M    Trainable params
0         Non-trainable params
29.3 M    Total params
117.180   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
Model_50_32x4d = model_return('/content/drive/MyDrive/DAI_Assignment_2/model3_50_32x4d/lightning_logs/version_0/checkpoints/epoch=99-step=19600.ckpt', ResNext_Model(model_variant='50_32x4d',num_classes=10,w1=0.7,w2=0.3))
detect(test_CIFAR10, Model_50_32x4d)
del Model_50_32x4d

100%|██████████| 10000/10000 [01:14<00:00, 134.22it/s]

              precision    recall  f1-score   support

           0       0.72      0.77      0.75      1000
           1       0.80      0.78      0.79      1000
           2       0.56      0.64      0.60      1000
           3       0.46      0.54      0.50      1000
           4       0.67      0.55      0.60      1000
           5       0.65      0.53      0.58      1000
           6       0.72      0.78      0.75      1000
           7       0.79      0.69      0.73      1000
           8       0.78      0.81      0.79      1000
           9       0.73      0.76      0.74      1000

    accuracy                           0.68     10000
   macro avg       0.69      0.68      0.68     10000
weighted avg       0.69      0.68      0.68     10000

TPR_PER_CLASS =  [0.767 0.778 0.644 0.542 0.549 0.529 0.775 0.686 0.807 0.759]
FPR_PER_CLASS =  [0.03244444 0.02133333 0.05622222 0.06944444 0.03033333 0.032
 0.03277778 0.02011111 0.025      0.03188889]
PPV_PER_CLASS =  [0.72426818 0.802061